In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
# from catboost import Pool, CatBoostRegressor
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, RobustScaler,PowerTransformer,QuantileTransformer,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [ ]:
# import data
train = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv',index_col = 0)
test = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv',index_col = 0)

In [ ]:
train.head()

In [ ]:
#Split the target and featues
y = train['loss']
f = train.drop(['loss'],axis = 1)

In [ ]:
y.head()

In [ ]:
f.dtypes

In [ ]:
#Check if we have any catagorical columns
cat_col = [c for c in f.columns if f[c].dtype =='object']
print(f"number of catagorical features {len(cat_col)}")
#check number of numeric features
num_col = [c for c in f.columns if f[c].dtype in ['int64','float64']]
print(f"number of catagorical features {len(num_col)}")

In [ ]:
#Make a copy of the original data
X = f[num_col].copy()
X_test = test[num_col].copy()

In [ ]:
#Data preprocessing
preprocessor = ColumnTransformer([('num',StandardScaler(),num_col)])

In [ ]:
#Train test split
X_t,X_v,y_t,y_v = train_test_split(X,y, train_size = 0.8,test_size = 0.2 ,random_state =10)

In [ ]:
# #Find the best model by trying value in the parameter tuning
# # l = list(np.arange(0.0,1,0.05))
# l = list(range(0,100,5))
# all_value={}
# for n in l:
#     model = XGBRegressor(
# #             verbosity = 0,
#             n_estimators = 5000,
#             max_debth = 3,#5,
#             learning_rate = 0.05,#0.12,
#             n_jobs = -1, #10,
#             random_state = 95,#10, #30,
#             colsample_bytree = 0.4,#0.85,
#             subsample = 0.85,#0.6,#0.9,
#             booster='gbtree',
#             reg_lambda= 50,#30,
#             reg_alpha=30,
#             tree_method = 'gpu_hist', # optimizer
#             predictor = 'gpu_predictor', #Make prediction faster
#             gamma = n
#         )

#     #Define Pipeline
#     pipe = Pipeline(steps = [('preprocessor',preprocessor),('model',model)])

#     pipe[0].fit(X_t)

#     fit_param = {"model__eval_set":[(pipe[0].transform(X_v),y_v)],
#                                     "model__early_stopping_rounds":7}
#     pipe.fit(X_t,y_t,**fit_param)
#     predict_value = pipe.predict(X_v)
#     all_value[n]=mean_squared_error(predict_value,y_v)
#     print("Mean Square Error:", mean_squared_error(predict_value,y_v))

In [ ]:
# min(all_value, key = all_value.get)

0.05: 61.09175065659344,\n
95: 61.01755668117088 \n
0.4: 61.003202057629714,
0.8500000000000001: 61.003202057629714,

In [ ]:
# all_value

In [ ]:
model = XGBRegressor(
#             verbosity = 0,
        n_estimators = 5000,
        max_debth = 3,#5,
        learning_rate = 0.05,#0.12,
        n_jobs = -1, #10,
        random_state = 95,#10, #30,
        colsample_bytree = 0.4,#0.85,
        subsample = 0.85,#0.6,#0.9,
        booster='gbtree',
        reg_lambda= 50,#30,
        reg_alpha=30,
#         tree_method = 'gpu_hist', # optimizer
#         predictor = 'gpu_predictor', #Make prediction faster
#         gamma = n
    )
#Define Pipeline
pipe = Pipeline(steps = [('preprocessor',preprocessor),('model',model)])

pipe[0].fit(X_t)

fit_param = {"model__eval_set":[(pipe[0].transform(X_v),y_v)],
                                "model__early_stopping_rounds":7}
pipe.fit(X_t,y_t,**fit_param)
predict_value = pipe.predict(X_v)
# all_value[n]=mean_squared_error(predict_value,y_v)
print("Mean Square Error:", mean_squared_error(predict_value,y_v))

In [ ]:
predict_test = pipe.predict(X_test)
predict_test

In [ ]:
output = pd.DataFrame({'id': X_test.index,
                       'loss': predict_test})
output.to_csv('submission.csv', index=False)

In [ ]:
output